<a href="https://colab.research.google.com/github/PanoEvJ/summarization_RLHF/blob/main/rlhf_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/peft.git

!pip install -q transformers==4.30
!pip install -q -U sentencepiece
!pip install -q huggingface_hub
!pip install -q tdqm torch>=0.3.0
!pip install -q -U bitsandbytes
!pip install -q -U wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Insta

In [4]:
import os
import random
import wandb

from dataclasses import dataclass, field
from typing import Optional

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset as TorchDataset
from datasets import load_dataset, Dataset
from peft import (AutoPeftModelForCausalLM,
                  LoraConfig,
                  PeftConfig,
                  PeftModel,
                  TaskType
)
from tqdm import tqdm
from transformers import (AutoModelForSeq2SeqLM,
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          AutoTokenizer,
                          T5ForConditionalGeneration,
                          T5Tokenizer
)
from trl import (SFTTrainer,
                 PPOConfig,
                 PPOTrainer,
                 AutoModelForSeq2SeqLMWithValueHead,
                 create_reference_model,
                 set_seed
)
from trl.core import LengthSampler

## Set Cuda

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

## Load the dataset

In [ ]:
from datasets import get_dataset_split_names

dataset_name = 'CarperAI/openai_summarize_comparisons'
get_dataset_split_names(dataset_name)

dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='train').shuffle(seed=42)
ppo_dataset = Dataset(dataset['prompt'])

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/92534 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/83629 [00:00<?, ? examples/s]

Generating valid1 split:   0%|          | 0/33082 [00:00<?, ? examples/s]

Generating valid2 split:   0%|          | 0/50715 [00:00<?, ? examples/s]

## Load the Rewards Model from Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
rewards_model_directory = "./drive/MyDrive/rewards_model/"

rm_model = AutoModelForSequenceClassification.from_pretrained(rewards_model_directory)
rm_tokenizer = AutoTokenizer.from_pretrained(rewards_model_directory)
rm_model.to(device)

In [18]:
import torch.nn.functional as F


def score_summaries(model, tokenizer, chosen_summary, rejected_summary):
    # Tokenize the inputs
    chosen_tokens = tokenizer(chosen_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    rejected_tokens = tokenizer(rejected_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

    chosen_tokens.to(device)
    rejected_tokens.to(device)

    # Get logits from the model
    with torch.no_grad():
        chosen_logits = model(**chosen_tokens).logits
        rejected_logits = model(**rejected_tokens).logits

    # Apply softmax to get probabilities
    chosen_probs = F.softmax(chosen_logits, dim=-1)
    rejected_probs = F.softmax(rejected_logits, dim=-1)

    # Assuming the positive class (indicating 'chosen' is good) is the second one
    chosen_score = chosen_probs[0][1].item()
    rejected_score = rejected_probs[0][1].item()

    # Extract logits for each summary
    chosen_logit = chosen_logits[0][1].item()
    rejected_logit = rejected_logits[0][1].item()

    return chosen_score, rejected_score, chosen_logit, rejected_logit

In [19]:
chosen_summary = "Water meter in another condo is not in our condo. What can we do legally to restore water to my condo complex?"
rejected_summary = "Go fix the problem."

In [20]:
chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

Chosen Score: 0.4960
Rejected Score: 0.5220
Chosen Logit: -0.0216
Rejected Logit: 0.0866


In [ ]:
delen(dataset)

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_use_double_quant=True,
#     bnb_8bit_quant_type="nf8",
#     bnb_8bit_compute_dtype=torch.bfloat16,
# )

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    # target_modules=["q", "k", "v"],
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    inference_mode=False,
)

In [ ]:
ref_model_id = 't5-base'
policy_model_id = 'PanoEvJ/T5_base_SFT_summarization'

ref_model = AutoModelForSeq2SeqLM.from_pretrained(ref_model_id,
                                                  device_map="auto",
                                                  load_in_8bit=True,
                                                  # quantization_config=bnb_config,
                                                 )
policy_model = AutoModelForSeq2SeqLM.from_pretrained(policy_model_id,
                                                     device_map="auto",
                                                     load_in_8bit=True,
                                                    #  quantization_config=bnb_config,
                                                    #  peft_config=peft_config
                                                    )

In [ ]:
# policy_model = PeftModel(policy_model, peft_config)

In [ ]:
# see the available modules by printint out the model
print(policy_model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(ref_model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(ref_model_id,
                                          model_max_length=512,
                                          truncation=True,
                                          padding=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ppo_config = PPOConfig(
    steps=512,
    model_name=policy_model,
    learning_rate=1e-4,
    batch_size=128,
    mini_batch_size=16,
    gradient_accumulation_steps=1,
    optimize_cuda_cache=True,
    ppo_epochs=8,
    target_kl=0.1
)

In [81]:
ppo_trainer = PPOTrainer(
    config= ppo_config,
    model=policy_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
)
dummy_dataloader = ppo_trainer.dataloader

ValueError: ignored

In [ ]:
for query_tensor, response_tensor in dummy_dataloader:
    # define a reward for response
    # (this could be any reward such as human feedback or output from another model)
    reward = [torch.tensor(1.0), torch.tensor(0.0)]
    # train model
    train_stats = ppo_trainer.step([q for q in query_tensor], [r for r in response_tensor], reward)
    break

In [ ]:
    question_tensors = batch["input_ids"]

    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )
    batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    # Compute reward score (using the sentiment analysis pipeline)
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"] - script_args.reward_baseline) for output in pipe_outputs]

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if epoch >= config.total_ppo_epochs:
        break


In [ ]:
# Run PPO step
stats = ppo_trainer.step(question_tensors, response_tensors, rewards)

In [ ]:
# Log stats to Wandb
ppo_trainer.log_stats(stats, batch, rewards)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./rewards_model/")
tokenizer = AutoTokenizer.from_pretrained("./rewards_model/")

NameError: ignored